# Recall, the model for linear regression:

\begin{align}
\vec{y} = f(\vec{w}) = \sum_{i=1}^{m} w_i x_i
\end{align}

and the task is to estime the least squares error (LSE) regression coeffficient $w_i$ for predicting the output $\vec{y}$ based on the observed data $X$, using gradient descent.


1. Normalize the features in the training data using $z$-score normalization.

2. Initialize the weights for the gradient descent algorithm all zeros i.e.
\begin{align}
\vec{w} = 
\begin{bmatrix}
0\\
0\\
0\\
\vdots\\
0
\end{bmatrix}
\end{align}

3. Using the following data set of parameters:

  * **Housing**: learning rate $= 0.4 \times 10^{-3}$, tolerance $= 0.5 \times 10^{-2}$.

  * **Yacht**: learning rate $= 0.1 \times 10^{-2}$, tolaerance $=0.1 \times 10^{-2}$

  * **Concrete**: learning rate $= 0.7 \times 10^{-3}$, tolerance $ = 0.1 \times 10^{-3}$.


Note: Here tolerance is defined based on the differenmt in root mean squared error (RMSE) measured on the training set between successive iterations. Where, RMSE is defined as 

\begin{align}
RMSE = \sqrt{\frac{SSE}{N}}
\end{align}

$N$ is the number of training instances.

4. Be sure to set a maximum number of interations (recommended maximum iterations = 50000) so that the algorithm does not run forever.

# **Normalization**:

In the previous assignment we normalized the entire dataset prior to learning, and then used ten-fold cross validation to evaluate the performance of the learning algorithm. However, the correct way of normalizing data would be to normalize the training data and record the normalize paraters (i.e. mean and standard deviation) for $z$-score normalization and min and max feature values for feature rescaling. This minimizes the chances of introducing any bias buring the performance evaluation of the learning algorithm. In  a real work scenario you would not have access to test data during the training phase.

To **summarize**: only nomralize your training data and then use nomalization parameters to normalize your test data and then estimate the accuracy/error.

**Adding the Constant Feature**: For every regression problem remember to add a column of ones to your dataset.

# **Least Regression using Normal Equations**

Use the `housing` and `yacht` data set to estimate the regression using normal equation. Contrast the performance (measured through RMSE) to the results obtained using gradient descent algorithm. In this problem, you will calculate the analytic solution that we obtained through Noraml Equations to learn your weight vector, and constrast the performance (training and test RMSE) for your graident-descent based impmentation for problem-1.

Students should add the following functions to their model and verify the results:

1. Add **regularization** term using $\|\vec{\theta}\|_{2}^{2}$ to:
  * **Normal equation** (closed form solution) [5 points]
  * **Gradient Descent** [5 points]
  * Be sure to pass the **regularization** as a **parameter** to your **class**, so you could run your model **with** regularization or **without** regularization term (version we saw in class)

2. Add a function to calculate the **Stochastic gradient descent** [10 points]
  * Be sure to pass the necessary **parameters** to your class so you could switch between **Gradient descent** or **stochastic gradient descent**.

3. Add a plotting function that will plot the error costs of the gradient descent. 

    * Be sure to plot the cost of your model during the training steps.

Report the RMSE and SSE over the test set for all three datasets.

Describe your observation and what are the effects of the learning rate and regularization parameters on the learning.

In [ ]:
# import Libraries 
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
class LinearRegression:
  def __init__(self, X, y, learningRate, Error, sample_size, 
               epsilon, maxIteration, lam, sgd = False, gd = False, reg = False) -> None:
    self.X = X # The default value for X is to let user input the prediction data 
    self.y = y # The default value for y is to let user input the target variable
    self.learningRate = learningRate # The default value for learning rate is to let user input the learning rate for gradient descient
    self.Error = Error # The input value for error is to let user descide wich type of error they want to input 
    self.sample_size = sample_size # The default value for sample_size is to let user decide the sample size for stochastic gradient 
    self.epsilon = epsilon # the default value for epsilon is to let user input the tolarence for gradient descient 
    self.maxIteration = maxIteration # The default value for maximum iteration for user input
    self.lam = lam # Let user input regularized parameter 
    self.sgd = sgd # let user input if they want to use the stochastic gradient; default is false 
    self.gd = gd # let user input if they want to use the gradient descent; default is false
    self.reg = reg # Let user input if they want to use the regularization; defualt is false 


  ##Define a function to split original data into test portion and training portion
  def splitData(self):
    X_train, X_test, Y_test, Y_train = train_test_split(self.X, self.y, 
                                                        test_size = 0.3, 
                                                        random_state = 0)
    return X_train, X_test, Y_test, Y_train

  ## Define a function to add a 0 column to the data as coefficient for linear regression
  def addX0(self, X):
    return np.column_stack([np.ones(X.shape[0]), X])

  ## Define a function to normalize the training data 
  def normalizeData(self, X):
    mean = np.mean(X, 0) # find the mean of each variable
    std = np.std(X,0) # find the standard deviation of each variable 
    X_norm = (X - mean) / std # using nomalization formula to normalize the data
    X_norm = self.addX0(X_norm) # adding 0 coulumn after normalize the data
    return X_norm, mean, std
  
  ## Define a function to normalize the training data set using the scalar define for training data set
  def normalizeTestData(self, X, mean, std):
    X_norm = (X - mean) / std # normalize the data
    X_norm = self.addX0(X_norm)  # adding zero column after normalize the data
    return X_norm

  ## Define a function to check if the data if full rank
  def checkFullRank(self, X):
    rank = np.linalg.matrix_rank(X) # Find the rank of the data 
    if rank == min(X.shape[0], X.shape[1]): # the rank should be equal to the min(#number of rows of the data, number of columns of the data)
      print('Data is Full Rank') # If it is true, print the data is full rank 
      self.Fullrank = True
    else: # other wise
      print('Data is Not Full Rank') # print the data is not full rank 
      self.Fullrank = False

  ## Define a function to check if the data is low rank
  def checkLowRank(self, X):
    if X.shape[0] < X.shape[1]: # if the number of rows is less than  then number of columns 
      print('Data is Low Rank') # print the data is low rank 
      self.lowRank = True
    else: # other wise
      print('Data is not Low Rank') # The data is not low rank 
      self.lowRank = False

  ## Define a function for normal equation for basic linear regression
  def normalEquation(self, X, y):
    w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    return w

  ## Define a function to predict the target variable
  def predict(self, X):
    y_hat = X.dot(self.w)
    return y_hat

  ## Define a function for normal equation for regularized linear regression 
  def normalReg(self, X, y):
    X_dim = self.addX0(X) # adding 0 column to the data set 
    dim = X_dim.shape[1] # find the dimension of the data set 
    I = np.identity(dim - 1) # Determine the identity matrix
    w = np.linalg.inv((X.T).dot(X) + self.lam * I).dot(X.T).dot(y)
    return w

  ## Define a function to calculate the sum squared error 
  def sse(self, X, y):
      y_hat = self.predict(X)
      return ((y_hat - y)**2).sum()

  ## Define a function to calculate the cost function 
  def costFunction(self, X, y):
      return self.sse(X, y) / 2 # Which is sse /2 (dividing 2 here is for easier calclulation)

  # Define a function for cost derivative
  def costDerivative(self, X, y):
      return X.T.dot(self.predict(X) - y)

  # Define a function for root mean squared error 
  def rmse(self, X, y):
    return math.sqrt(self.sse(X, y)/y.size)

  ## Define a function for gradient descent for basic linear regression 
  def gradientDescent(self, X, y):
    errors = [] # Define an empty list for collecting the errors
    prev_error = float('inf')  # suppose that previous error is very large 
    self.costValues = [] # define a function to collecte the cost values 
    for i in tqdm(range(self.maxIteration)): # loop the iteration from using tqdm function 
      self.w = self.w - self.learningRate * self.costDerivative(X, y) # use the gradient descent method where costDerivative is the gradient 
      cost = self.costFunction(X, y) # The cost is the costFunction for X and y
      self.costValues.append(cost) # update the costValue list 
      if self.Error == 'rmse': # If the user wants to report rmse
        current_error = self.rmse(X, y) # find the rmse
      else: # other wise
        current_error = self.sse(X, y) # go for sse
      diff = current_error - prev_error # the error is defind as current error - prev error
      errors.append(current_error) # update the error list with current error 
      prev_error = current_error # swap prev error and current error 
      if abs(diff) < self.epsilon: # if the abs or current error - prev error is less than the desired tolarence
        print('Model is stopped learning.') # The model is stopped learning 
        break
    self.plot_rmse(errors)

  # Define a function for calculate the cost derivative for regularized linear regression 
  def costDerivativeReg(self, X, y):
    y_hat = self.predict(X)
    return (y_hat - y).dot(X) + self.lam*(self.w)

  # Define a gradient descent method for regularized linear regression 
  def gdReg(self, X, y):
    errors = [] # Define an empty list for collecting the errors 
    self.costValues = [] # Define an empty list for colletecting the cost valus 
    prev_error  = float('inf') # suppse the previous error is very big 
    for i in tqdm(range(self.maxIteration)): # loop the iteration using tqdm function 
      self.w = self.w - self.learningRate*self.costDerivativeReg(X, y) # use the graident descient method for regularized linear regression 
      cost = self.costFunction(X, y) # The cost value is the cost function 
      self.costValues.append(cost) # update the cost value
      if self.Error == 'rmse': # If the user wants to repost the rmse 
        current_error  = self.rmse(X, y) # current error is the rmse
      else: # otherwise
        current_error = self.sse(X, y) # the current error is sse
      diff = current_error - prev_error # The error is current error - prev error 
      prev_error = current_error    # swap prev and current error 
      errors.append(current_error) # update the error list 
      if abs(diff) < self.epsilon: # If the abs(current and prev errors) < desired tol
        print('Model is stopped learning') # model is stopped learning
        break
    self.plot_rmse(errors)

  # Deinfe a function for stochastic gradient
  def stochGD(self, X, y):
    errors = [] # Define an empty list to collect the errors 
    self.costValues = [] # Define an empty list for cost values
    prev_error = float('inf') # Suppse that prev error is very large
    for i in tqdm(range(self.maxIteration)): # Loop the iteration using tqdm function 
      b = random.sample(list(X), self.sample_size) # Find the sample size for X 
      c = random.sample(list(y), self.sample_size) # Find the sample size for y
      y = np.array(c) # Transfer the y to a vector
      X = np.array(b) # transfer the X to a matrix
      for q,r in zip(X, y): # Define the set of points for (X,y)
        self.w = self.w - self.learningRate*self.costDerivative(X, y) # Using the gradient descent for basic linear regression 
        cost = self.costFunction(X, y) # Find the cost values
        self.costValues.append(cost) # update the cost values 
        if self.Error == 'rmse': # If the user wants to report the rmse
          current_error = self.rmse(X, y) # The current error is rmse
        else: #otherwise
          current_error = self.sse(X, y) # The current error is sse
        diff = current_error - prev_error # the error is current_error - prev_error 
        prev_error = current_error # swap prev and current errors      
        errors.append(current_error) # update the error list
        if abs(diff) < self.epsilon: # if the abs(error) < desired tol 
          #print('Model is stopped learning') #Model is topped learning 
          break      
    self.plot_rmse(errors)

  # Define a function for stochastic gradicent for regularized linear regression 
  def stochGdReg(self, X, y):
    errors= [] # Define an empty list for errors 
    self.costValues = [] # Define an empty list for cost values 
    prev_error = float('inf')  # Suppse that prev error is very large
    for i in tqdm(range(self.maxIteration)): # Loop the iteration using tqdm function
      b = random.sample(list(X), self.sample_size) # Find the sample size for X 
      c = random.sample(list(y), self.sample_size) # Find the sample size for y
      y = np.array(c) # Transfer the y to a vector
      X = np.array(b) # transfer the X to a matrix
      for q, r in zip(X, y): # Define the set of points for (X,y)
        self.w = self.w - self.learningRate*self.costDerivativeReg(X,y) # use the graident descient method for regularized linear regression
        cost = self.costFunction(X, y) # update the cost values
        self.costValues.append(cost) # update the cost values 
        if self.Error == 'rmse': #If the user wants to report the rmse
          current_error = self.rmse(X, y) # The current error is rmse
        else: #otherwise
          current_error = self.sse(X,y)     # The current error is sse 
        diff = current_error - prev_error # the error is current_error - prev_error
        prev_error = current_error     # Swap the previous error and current error 
        errors.append(current_error)    # Update the error lisr with current errors        
        if abs(diff) < self.epsilon: # If the abs(current error - prev error) < a desired tol 
          #print('Model is stopped learning.') # Model will stop learning
          break
      plt.plot(errors)

  # Define a function to plot the room mean sqaured error 
  def plot_rmse(self, error_sequence):
    # Data for plotting
    s = np.array(error_sequence)
    t = np.arange(s.size)

    fig, ax = plt.subplots()
    ax.plot(t, s)

    ax.set(xlabel='Iterations', ylabel = 'Error',
           title='Iterations andError Trend')
    ax.grid()
    plt.legend(bbox_to_anchor=(1.05,1), loc=2, shadow=True)
    plt.show()
  
  # Define a function for fit 
  def fit(self):
    self.X_train, self.X_test, self.y_train, self.y_test = self.splitData() # Data spliting 
    self.X_train, self.mean, self.sd = self.normalizeData(self.X_train) # Normalize the  traing data 
    self.X_test = self.normalizeTestData(self.X_test, self.mean, self.sd) # Normalize the test data
    self.checkFullRank(self.X_train) # Check the full rank 
    self.checkLowRank(self.X_train)  # Check the low rank 

    # If the data is full rank it is not no rank . If the gradience descent, regularization, and stochastic gradient descent
      ## are not going to be used  
    if self.Fullrank and not self.lowRank and not self.gd and not self.reg and not self.sgd: 
      # go for closed form solution for basic linear regression   
      print('Closed Form Solution for Basic Linear Regression')
      self.w = self.normalEquation(self.X_train, self.y_train)

    # if the data is full rank, not low rank, gradient descent is not going to be used, stochatic gradient is not going to be used
      ## regularization is going to be used           
    elif self.Fullrank and not self.lowRank and not self.gd and self.reg and not self.sgd:
      # go for closed form solution for regularizaed linear regression
      print('Closed Form Solution Regularization Linear Regression')
      self.w = self.normalReg(self.X_train, self.y_train)

    # If the gradient is going to be used and regularization is going to be used and stochastic gradient descent is not going to be used:
    elif self.gd and self.reg and not self.sgd:
      # go for gradient descent for regularized linear regression
      print('Gradient Descent for Regularization Linear Regression')
      #Initialize your weights
      self.w = np.zeros(self.X_train.shape[1]) #initialize weights to zeros
      self.gdReg(self.X_train, self.y_train)
    
    # If the stochasich gradient is going to be used, if the regularization is going to be used
      ## if the graident is not going to be used
    elif self.sgd and self.reg and not self.gd:
      # go for stochastic gradient descent for regularized linear regression
      print('Stochastic Gradient Descent for Regularization Linear Regression')
      # Initialize your weights
      self.w = np.zeros(self.X_train.shape[1]) #initialize weights to zeros
      self.stochGdReg(self.X_train, self.y_train)
    
    # If stochastic gradient is going to be used, regularization is not going to be used, as well as gradient descient is not going to be used
    elif self.sgd and not self.reg and not self.gd:
      # Go for stochastic gradient descent 
      print('Stochastic Gradient Descent')
      # Initialize your starting weights
      self.w = np.zeros(self.X_train.shape[1]) # Initialize weights to zeors
      self.stochGD(self.X_train, self.y_train)

    # Else
    else:
      # Use the gradient descent for basic linear regression
      print("Gradient Descent for Basic Linear Regression")
      # Initialize your weights
      self.w = np.zeros(self.X_train.shape[1]) # Initialize weights to all zeors 
      self.gradientDescent(self.X_train,self.y_train)

    sse_test = self.sse(self.X_test, self.y_test) # Find the sum of squared error on testing data
    rmse_test = self.rmse(self.X_test, self.y_test) # Find the root mean squared error on testing data
    sse_train = self.sse(self.X_train, self.y_train) # Find the sum of squared error on training data 
    rmse_train = self.rmse(self.X_train, self.y_train) # Find the sse on training data
    print("The SSE for test data is {0}". format(round(sse_test,2))) # Primt the SSE and round your result to 2 decimals 
    print("The RMSE for test data is {0}". format(round(rmse_test,2))) # Print the RMSE and round your result to 2 decimals
    print("The SSE for training data is {0}". format(round(sse_train,2)))
    print("The RMSE for training data is {0}". format(round(rmse_train,2)))
    print("Weights:") 
    print(self.w) # Print the weight 

## **Housing data**: 

This is a regression dataset where the task is to predict the value of houses in the suburbs of Boston based on thirteen features that describe different aspects that are relevent to determining the value of a house, such as the number of rooms, levels of population in the area, etc.

In [ ]:
# Import the housing data set
df1 = pd.read_csv("Project Dataset.csv")
df1

In [ ]:
df1['Runtime']=df1[['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)']].mean(axis=1)
df1=df1.drop(columns =['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)'], axis = 1)

In [ ]:
df1

In [ ]:
X = df1.values[:,0:-1]

In [ ]:
y = df1.values[:,-4]

Housing data

In [ ]:
# Apply the normal equation on Housing data set without regularization
clf1 = LinearRegression(df1.values[:,0:-1], 
                              df1.values[:,-1], sample_size = 100, lam = 0.001, learningRate = 0.0004, Error = 'rmse',
                        epsilon = 0.005, maxIteration = 500, sgd=False, gd = False, reg = False)
clf1.fit()

In [ ]:
# Apply the gradient descent on Housing data set without regularization
clf2 = LinearRegression(df1.values[:,0:-1], 
                              df1.values[:,-1], sample_size = 100, lam = 0.0001, learningRate = 0.0004, Error = 'rmse',
                        epsilon = 0.005, maxIteration = 50000, sgd=False, gd = False, reg = True)
clf2.fit()

In [ ]:
# Apply the basic gradient descent on Housing data set without regularization
clf3 = LinearRegression(df1.values[:,0:-1], 
                              df1.values[:,-1], sample_size = 100, lam = 0.0001, learningRate = 0.0004, Error = 'rmse',
                        epsilon = 0.005, maxIteration =500, sgd=False, gd = True, reg = False)
clf3.fit()

In [ ]:
# Apply the regularized gradient descent on Housing data set without regularization
clf4 = LinearRegression(df1.values[:,0:-1], 
                              df1.values[:,-1], sample_size = 100, lam = 0.0001, learningRate = 0.0004, Error = 'rmse',
                        epsilon = 0.005, maxIteration = 500, sgd=False, gd = True, reg = True)
clf4.fit()

In [ ]:
# Apply the stochastic gradient descent on Housing data set 
clf5 = LinearRegression(df1.values[:,0:-1], 
                              df1.values[:,-1], sample_size = 200, lam = 0.0001, learningRate = 0.0004, Error = 'rmse',
                        epsilon = 0.005, maxIteration = 50, sgd=True, gd = False, reg = False)
clf5.fit()

In [ ]:
# Determine the root mean squared error for housing data used by normal equation
RMSE_Housing = clf1.rmse(clf1.X_test, clf1.y_test)
print("The RMSE for hosing data on test portion is {0}".format(RMSE_Housing))